In [1]:
import findspark
findspark.init()
findspark.find()

'C:\\Users\\Admin\\anaconda3\\envs\\SparkEnvironment\\Lib\\site-packages\\pyspark'

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = (
    SparkSession
    .builder
    .appName("SparkTablesApp")
    .master("local[4]")
    .config("spark.dynamicAllocation.enabled", "false")
    .config("spark.sql.adaptive.enabled", "false")
    .enableHiveSupport()
    .getOrCreate()
)

sc = spark.sparkContext

spark

In [3]:
# Create schema for Yellow Taxi Data
 
taxiSchema = (
                    StructType
                    ([ 
                        StructField("VendorId"               , IntegerType()   , True),
                        StructField("lpep_pickup_datetime"   , TimestampType() , True),
                        StructField("lpep_dropoff_datetime"  , TimestampType() , True),                            
                        StructField("passenger_count"        , DoubleType()    , True),
                        StructField("trip_distance"          , DoubleType()    , True),
                        StructField("RatecodeID"             , DoubleType()    , True),                            
                        StructField("store_and_fwd_flag"     , StringType()    , True),
                        StructField("PULocationID"           , IntegerType()   , True),
                        StructField("DOLocationID"           , IntegerType()   , True),                            
                        StructField("payment_type"           , IntegerType()   , True),                            
                        StructField("fare_amount"            , DoubleType()    , True),
                        StructField("extra"                  , DoubleType()    , True),
                        StructField("mta_tax"                , DoubleType()    , True),
                        StructField("tip_amount"             , DoubleType()    , True),
                        StructField("tolls_amount"           , DoubleType()    , True),
                        StructField("improvement_surcharge"  , DoubleType()    , True),
                        StructField("total_amount"           , DoubleType()    , True),
                        StructField("congestion_surcharge"   , DoubleType()    , True),
                        StructField("airport_fee"            , DoubleType()    , True)
                    ])
               )

In [4]:
yellowTaxiDf = (
    spark
    .read
    .option("header", "true")
    .schema(taxiSchema)
    .csv("C:\DataFiles\YellowTaxis_202210.csv")
)
yellowTaxiDf.printSchema()

root
 |-- VendorId: integer (nullable = true)
 |-- lpep_pickup_datetime: timestamp (nullable = true)
 |-- lpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)



In [5]:
spark.sql("""
show databases
""").show()

+---------+
|namespace|
+---------+
|  default|
+---------+



In [8]:
spark.sql("""
create database if not exists TaxisDB
""")

DataFrame[]

In [12]:
spark.sql("""
show databases
""").show()

+---------+
|namespace|
+---------+
|  default|
|  taxisdb|
+---------+



In [10]:
(
    yellowTaxiDf
    .write
    .mode("overwrite")
    .saveAsTable("TaxisDB.YellowTaxisManaged")
)

In [11]:
spark.sql("""
show tables in TaxisDB
""").show(50, truncate = False)

+---------+------------------+-----------+
|namespace|tableName         |isTemporary|
+---------+------------------+-----------+
|taxisdb  |yellowtaxismanaged|false      |
+---------+------------------+-----------+



In [13]:
(yellowTaxiDf
 .write
 .mode("overwrite")
 .option("path", "C:\DataFiles\output.parquet")
 .saveAsTable("TaxisDB.YellowTaxis")
)

In [14]:
spark.sql("""
describe table extended TaxisDB.Yellowtaxis
""").show(50, truncate = False)

+----------------------------+--------------------------------------------------------------+-------+
|col_name                    |data_type                                                     |comment|
+----------------------------+--------------------------------------------------------------+-------+
|VendorId                    |int                                                           |NULL   |
|lpep_pickup_datetime        |timestamp                                                     |NULL   |
|lpep_dropoff_datetime       |timestamp                                                     |NULL   |
|passenger_count             |double                                                        |NULL   |
|trip_distance               |double                                                        |NULL   |
|RatecodeID                  |double                                                        |NULL   |
|store_and_fwd_flag          |string                                              

In [15]:
spark.sql("""
drop table TaxisDB.yellowTaxis
""")


DataFrame[]

In [16]:
spark.sql("""
create table TaxisDB.yellowTaxis
using parquet
location "C://DataFiles//output.parquet"
""")

DataFrame[]